# QC-Py-02 : QuantConnect Platform Fundamentals - QCAlgorithm Lifecycle

## Introduction

Dans ce notebook, nous allons approfondir les fondamentaux de la plateforme QuantConnect en explorant le cycle de vie de la classe `QCAlgorithm`, qui constitue le coeur de toute stratégie de trading algorithmique sur QuantConnect.

### Récapitulatif QC-Py-01

Dans le notebook précédent, nous avons :
- Créé un compte QuantConnect
- Exploré l'interface web
- Créé et exécuté notre premier algorithme "Hello World"
- Compris la différence entre backtest et live trading

### Objectifs de ce notebook

À la fin de ce notebook, vous serez capable de :

1. **Maîtriser le lifecycle QCAlgorithm** (Initialize, OnData, OnEndOfAlgorithm)
2. **Comprendre la gestion des securities** (AddEquity, AddCrypto, etc.)
3. **Créer des stratégies simples** avec indicateurs techniques intégrés
4. **Découvrir les principales méthodes** de QCAlgorithm

### Prérequis

- Notebook QC-Py-01 complété (compte QuantConnect créé)
- Python niveau intermédiaire
- Notions de base en trading (optionnel mais utile)

### Plan du notebook

1. **QCAlgorithm Lifecycle** (15 min)
2. **Gestion des Securities** (15 min)
3. **Indicateurs Techniques Intégrés** (15 min)
4. **Exemple Complet : RSI Mean Reversion** (15 min)
5. **Méthodes QCAlgorithm Essentielles** (10 min)
6. **Conclusion** (5 min)

**Durée estimée** : 60 minutes

## 2. QCAlgorithm Lifecycle

### Vue d'ensemble du cycle de vie

Toute stratégie QuantConnect hérite de la classe `QCAlgorithm` et implémente trois méthodes principales qui définissent le cycle de vie de l'algorithme :

```
┌─────────────────────────────────────────┐
│  BACKTEST START                         │
└─────────────────────────────────────────┘
               │
               ▼
┌─────────────────────────────────────────┐
│  Initialize()                           │
│  • Appelé UNE SEULE FOIS au début       │
│  • Configuration dates, capital         │
│  • Ajout des securities                 │
│  • Création des indicateurs             │
└─────────────────────────────────────────┘
               │
               ▼
┌─────────────────────────────────────────┐
│  OnData(data)                           │
│  • Appelé à CHAQUE nouvelle donnée      │
│  • Logique de trading                   │
│  • Décisions d'achat/vente              │
│  ◄────────────────────────┐             │
│                           │             │
│  (boucle principale)      │             │
└─────────────────────────────────────────┘
               │
               ▼
┌─────────────────────────────────────────┐
│  OnEndOfAlgorithm()                     │
│  • Appelé UNE FOIS à la fin             │
│  • Nettoyage                            │
│  • Logs finaux                          │
└─────────────────────────────────────────┘
               │
               ▼
┌─────────────────────────────────────────┐
│  BACKTEST END                           │
└─────────────────────────────────────────┘
```

### Les trois méthodes clés

| Méthode | Appels | Rôle | Exemple d'utilisation |
|---------|--------|------|----------------------|
| `Initialize()` | 1 fois (début) | Configuration initiale | Définir dates, ajouter SPY, créer SMA(20) |
| `OnData(data)` | À chaque tick/bar | Logique de trading | Si SMA croisé, acheter/vendre |
| `OnEndOfAlgorithm()` | 1 fois (fin) | Finalisation | Logger le rendement final |

### Structure de base d'un algorithme

Voici le squelette minimal de tout algorithme QuantConnect :

In [ ]:
# Structure de base d'un algorithme QuantConnect
# Note: Ce code est informatif, il ne peut pas s'exécuter localement
# Il doit être copié dans l'IDE QuantConnect

from AlgorithmImports import *

class BasicLifecycleAlgorithm(QCAlgorithm):
    
    def Initialize(self):
        """
        Appelé une fois au début du backtest.
        Configuration: dates, capital, securities, indicateurs.
        """
        # Définir la période de backtest
        self.SetStartDate(2020, 1, 1)
        self.SetEndDate(2023, 12, 31)
        
        # Définir le capital initial
        self.SetCash(100000)
        
        # Ajouter des actifs (securities)
        self.symbol = self.AddEquity("SPY", Resolution.Daily).Symbol
        
        self.Log("Algorithm initialized")
    
    def OnData(self, data):
        """
        Appelé à chaque nouvelle donnée (tick, minute, hour, daily).
        Logique de trading ici.
        """
        # Vérifier que les données sont disponibles
        if not data.ContainsKey(self.symbol):
            return
        
        # Accéder au prix actuel
        price = data[self.symbol].Close
        
        # Exemple: acheter si on n'est pas investi
        if not self.Portfolio.Invested:
            self.SetHoldings(self.symbol, 1.0)
            self.Log(f"Bought SPY at ${price:.2f}")
    
    def OnEndOfAlgorithm(self):
        """
        Appelé à la fin du backtest.
        Nettoyage, logs finaux.
        """
        final_value = self.Portfolio.TotalPortfolioValue
        self.Log(f"Algorithm ended. Final Portfolio Value: ${final_value:.2f}")

### Fréquence d'appel de OnData selon Resolution

La méthode `OnData` est appelée selon la résolution (Resolution) choisie lors de l'ajout de la security :

| Resolution | Fréquence d'appel | Exemple d'usage |
|------------|-------------------|------------------|
| `Resolution.Tick` | À chaque tick (millisecondes) | Scalping, market making |
| `Resolution.Second` | Toutes les secondes | Trading haute fréquence |
| `Resolution.Minute` | Toutes les minutes | Day trading |
| `Resolution.Hour` | Toutes les heures | Swing trading |
| `Resolution.Daily` | Tous les jours (9h30 ET) | Position trading |

**Important** : Plus la résolution est fine, plus le backtest est lent et consomme de ressources. Commencez avec `Resolution.Daily` pour vos tests initiaux.

### Debug avec self.Log() et self.Debug()

QuantConnect offre deux méthodes pour logger des informations :

```python
# self.Log() - Visible dans les résultats du backtest
self.Log("Message important qui apparaîtra dans le rapport")

# self.Debug() - Visible dans la console de debug (temps réel)
self.Debug(f"Prix actuel: {price}")
```

**Conseil** : Utilisez `self.Log()` pour des événements importants (achats, ventes) et `self.Debug()` pour du debug détaillé.

## 3. Gestion des Securities

### Qu'est-ce qu'une Security ?

Une **Security** (titre financier) représente un actif tradable sur QuantConnect. Avant de pouvoir trader un actif, vous devez l'ajouter à votre algorithme avec une méthode `Add*`.

### Principales classes d'actifs

QuantConnect supporte plusieurs classes d'actifs :

| Classe d'actif | Méthode | Exemple | Résolution recommandée |
|----------------|---------|---------|------------------------|
| **Actions (Equity)** | `AddEquity(ticker, resolution)` | `AddEquity("AAPL", Resolution.Daily)` | Daily/Minute |
| **Crypto** | `AddCrypto(pair, resolution)` | `AddCrypto("BTCUSD", Resolution.Hour)` | Hour/Minute |
| **Forex** | `AddForex(pair, resolution)` | `AddForex("EURUSD", Resolution.Minute)` | Minute/Second |
| **Options** | `AddOption(underlying)` | `AddOption("SPY")` | Minute (détaillé en QC-Py-06) |
| **Futures** | `AddFuture(ticker)` | `AddFuture("ES")` | Minute (détaillé en QC-Py-07) |

### Symbol vs String ticker

**Point clé** : Les méthodes `Add*` retournent un objet `Symbol`, pas une simple string.

```python
# ❌ MAUVAIS - utiliser directement le ticker
self.AddEquity("SPY", Resolution.Daily)
# Plus tard dans OnData...
price = data["SPY"].Close  # Peut causer des erreurs

# ✅ BON - stocker le Symbol
self.symbol_spy = self.AddEquity("SPY", Resolution.Daily).Symbol
# Plus tard dans OnData...
price = data[self.symbol_spy].Close  # Robuste
```

Le `Symbol` contient plus d'informations que le simple ticker (marché, type d'actif, etc.) et évite les ambiguïtés.

### Exemples d'ajout de securities

In [ ]:
# Exemples d'ajout de différentes classes d'actifs
# Note: Code informatif pour QuantConnect IDE

from AlgorithmImports import *

class MultiAssetAlgorithm(QCAlgorithm):
    
    def Initialize(self):
        self.SetStartDate(2020, 1, 1)
        self.SetEndDate(2023, 12, 31)
        self.SetCash(100000)
        
        # 1. Equity (actions)
        # SPY est un ETF très liquide qui réplique le S&P 500
        self.symbol_spy = self.AddEquity("SPY", Resolution.Daily).Symbol
        self.Log(f"Added SPY: {self.symbol_spy}")
        
        # 2. Crypto
        # BTCUSD = Bitcoin vs USD
        # Note: Résolution Hour car crypto trade 24/7
        self.symbol_btc = self.AddCrypto("BTCUSD", Resolution.Hour).Symbol
        self.Log(f"Added BTCUSD: {self.symbol_btc}")
        
        # 3. Forex
        # EURUSD = Euro vs US Dollar
        self.symbol_eur = self.AddForex("EURUSD", Resolution.Minute).Symbol
        self.Log(f"Added EURUSD: {self.symbol_eur}")
        
        # 4. Options (aperçu - détaillé dans QC-Py-06)
        # Ajoute les options disponibles sur SPY
        # self.option_spy = self.AddOption("SPY")
        # self.option_spy.SetFilter(-5, 5, 0, 30)  # Strike range, expiry days
        
        # 5. Futures (aperçu - détaillé dans QC-Py-07)
        # ES = E-mini S&P 500 futures
        # self.future_es = self.AddFuture("ES")
    
    def OnData(self, data):
        # Vérifier disponibilité des données pour chaque symbol
        if data.ContainsKey(self.symbol_spy):
            spy_price = data[self.symbol_spy].Close
            self.Debug(f"SPY: ${spy_price:.2f}")
        
        if data.ContainsKey(self.symbol_btc):
            btc_price = data[self.symbol_btc].Close
            self.Debug(f"BTC: ${btc_price:.2f}")

### Warm-up period avec SetWarmUp()

Les indicateurs techniques (SMA, EMA, RSI, etc.) nécessitent un certain nombre de données historiques avant d'être prêts ("ready"). Sans warm-up, les premiers jours du backtest n'auront pas d'indicateurs valides.

**Solution** : Utiliser `SetWarmUp()` pour charger des données historiques avant le début du backtest.

```python
def Initialize(self):
    self.SetStartDate(2020, 1, 1)
    self.SetCash(100000)
    
    self.symbol = self.AddEquity("SPY", Resolution.Daily).Symbol
    
    # Créer un SMA de 50 jours
    self.sma_50 = self.SMA(self.symbol, 50)
    
    # Warm-up de 50 jours pour remplir le SMA
    self.SetWarmUp(50)

def OnData(self, data):
    # Ignorer les données de warm-up
    if self.IsWarmingUp:
        return
    
    # Vérifier que l'indicateur est prêt
    if not self.sma_50.IsReady:
        return
    
    # À ce stade, le SMA est valide
    sma_value = self.sma_50.Current.Value
```

**Conseil** : Toujours utiliser `SetWarmUp()` avec un nombre égal à la période du plus long indicateur.

### Vérifier la disponibilité : data.ContainsKey(symbol)

Dans `OnData`, il faut toujours vérifier que les données sont disponibles pour un symbol donné :

```python
def OnData(self, data):
    # ❌ MAUVAIS - peut crasher si les données manquent
    price = data[self.symbol].Close
    
    # ✅ BON - vérifier d'abord
    if data.ContainsKey(self.symbol):
        price = data[self.symbol].Close
    else:
        return  # Pas de données, on attend la prochaine itération
```

## 4. Indicateurs Techniques Intégrés

### Bibliothèque d'indicateurs QuantConnect

QuantConnect intègre une bibliothèque complète d'indicateurs techniques. Plus besoin de coder les formules manuellement !

### Principaux indicateurs disponibles

| Indicateur | Code | Description | Utilisation typique |
|------------|------|-------------|---------------------|
| **SMA** | `self.SMA(symbol, period)` | Simple Moving Average | Tendance, support/résistance |
| **EMA** | `self.EMA(symbol, period)` | Exponential Moving Average | Tendance (plus réactif que SMA) |
| **RSI** | `self.RSI(symbol, period)` | Relative Strength Index | Surachat/survente (0-100) |
| **MACD** | `self.MACD(symbol, fast, slow, signal)` | Moving Average Convergence Divergence | Momentum, changements de tendance |
| **BB** | `self.BB(symbol, period, std_dev)` | Bollinger Bands | Volatilité, breakouts |
| **ATR** | `self.ATR(symbol, period)` | Average True Range | Mesure de volatilité |
| **STOCH** | `self.STO(symbol, period)` | Stochastic Oscillator | Momentum, retournements |

La liste complète : [QuantConnect Indicators Documentation](https://www.quantconnect.com/docs/v2/writing-algorithms/indicators/supported-indicators/overview)

### Créer et utiliser des indicateurs

In [ ]:
# Création d'indicateurs techniques dans Initialize()
# Note: Code informatif pour QuantConnect IDE

from AlgorithmImports import *

class IndicatorsAlgorithm(QCAlgorithm):
    
    def Initialize(self):
        self.SetStartDate(2020, 1, 1)
        self.SetEndDate(2023, 12, 31)
        self.SetCash(100000)
        
        self.symbol = self.AddEquity("SPY", Resolution.Daily).Symbol
        
        # 1. SMA (Simple Moving Average)
        # Moyenne mobile simple sur 20 jours
        self.sma_20 = self.SMA(self.symbol, 20, Resolution.Daily)
        
        # 2. EMA (Exponential Moving Average)
        # Moyenne mobile exponentielle sur 50 jours (plus réactive)
        self.ema_50 = self.EMA(self.symbol, 50)
        
        # 3. RSI (Relative Strength Index)
        # Oscillateur de momentum entre 0 et 100
        # Classiquement: RSI < 30 = survente, RSI > 70 = surachat
        self.rsi = self.RSI(self.symbol, 14)
        
        # 4. MACD (Moving Average Convergence Divergence)
        # fast=12, slow=26, signal=9 (paramètres standards)
        self.macd = self.MACD(self.symbol, 12, 26, 9)
        
        # 5. Bollinger Bands
        # period=20, std_dev=2 (paramètres standards)
        self.bb = self.BB(self.symbol, 20, 2)
        
        # Warm-up: utiliser la période du plus long indicateur
        self.SetWarmUp(50)
        
        self.Log("Indicators created successfully")
    
    def OnData(self, data):
        # Ignorer la période de warm-up
        if self.IsWarmingUp:
            return
        
        # Vérifier que tous les indicateurs sont prêts
        if not (self.sma_20.IsReady and self.ema_50.IsReady and 
                self.rsi.IsReady and self.macd.IsReady and self.bb.IsReady):
            return
        
        # Accéder aux valeurs des indicateurs
        sma_value = self.sma_20.Current.Value
        ema_value = self.ema_50.Current.Value
        rsi_value = self.rsi.Current.Value
        
        # MACD a plusieurs composantes
        macd_value = self.macd.Current.Value      # MACD line
        macd_signal = self.macd.Signal.Current.Value  # Signal line
        macd_hist = self.macd.Histogram.Current.Value  # Histogram
        
        # Bollinger Bands a 3 bandes
        bb_upper = self.bb.UpperBand.Current.Value
        bb_middle = self.bb.MiddleBand.Current.Value
        bb_lower = self.bb.LowerBand.Current.Value
        
        # Afficher les valeurs (debug)
        self.Debug(f"SMA(20): {sma_value:.2f} | EMA(50): {ema_value:.2f} | RSI: {rsi_value:.2f}")

### Vérifier si ready : indicator.IsReady

Un indicateur n'est **pas immédiatement prêt** après sa création. Il faut attendre qu'il ait reçu assez de données.

```python
# Créer un SMA(50)
self.sma_50 = self.SMA(self.symbol, 50)

# Dans OnData:
if self.sma_50.IsReady:
    value = self.sma_50.Current.Value  # ✅ Safe
else:
    self.Debug("SMA not ready yet")  # ⏳ Attendre
```

**Important** : Toujours vérifier `IsReady` avant d'accéder à la valeur, sinon vous risquez d'obtenir des valeurs incorrectes (NaN ou 0).

### Accéder à la valeur : indicator.Current.Value

Pour obtenir la valeur actuelle d'un indicateur :

```python
# Pour les indicateurs simples (SMA, EMA, RSI, ATR)
value = self.sma_20.Current.Value

# Pour les indicateurs complexes (MACD, BB, STOCH)
macd_line = self.macd.Current.Value
macd_signal = self.macd.Signal.Current.Value
macd_histogram = self.macd.Histogram.Current.Value

bb_upper = self.bb.UpperBand.Current.Value
bb_middle = self.bb.MiddleBand.Current.Value
bb_lower = self.bb.LowerBand.Current.Value
```

### Visualiser avec self.Plot()

QuantConnect permet de créer des graphiques personnalisés avec `self.Plot()` :

```python
def OnData(self, data):
    if not self.sma_20.IsReady:
        return
    
    price = data[self.symbol].Close
    
    # Créer un graphique avec le prix et le SMA
    self.Plot("Price vs SMA", "SPY Price", price)
    self.Plot("Price vs SMA", "SMA(20)", self.sma_20.Current.Value)
    
    # Créer un graphique séparé pour le RSI
    if self.rsi.IsReady:
        self.Plot("RSI", "RSI(14)", self.rsi.Current.Value)
        self.Plot("RSI", "Overbought", 70)  # Ligne de référence
        self.Plot("RSI", "Oversold", 30)    # Ligne de référence
```

Les graphiques apparaissent dans les résultats du backtest sous l'onglet "Charts".

## 5. Exemple Complet : RSI Mean Reversion

### Stratégie RSI Mean Reversion

Nous allons maintenant créer une stratégie complète basée sur le **RSI (Relative Strength Index)**, un oscillateur très populaire.

#### Principe de la stratégie

Le RSI mesure la force du momentum d'un actif sur une échelle de 0 à 100 :

- **RSI < 30** : Zone de **survente** → Le prix a trop baissé, possibilité de rebond (**signal d'achat**)
- **RSI > 70** : Zone de **surachat** → Le prix a trop monté, possibilité de correction (**signal de vente**)

Cette stratégie suppose que les prix ont tendance à **revenir vers la moyenne** (mean reversion) après des mouvements extrêmes.

#### Règles de trading

```
SI RSI < 30 ET pas investi :
    ACHETER 100% du capital

SI RSI > 70 ET investi :
    VENDRE toute la position
```

#### Diagramme de la logique

```
        RSI(14)
          │
     ┌────┴────┐
     │         │
   < 30      > 70
     │         │
 OVERSOLD  OVERBOUGHT
     │         │
    BUY      SELL
```

### Code complet de la stratégie

Copiez ce code dans l'IDE QuantConnect pour l'exécuter :

In [ ]:
# Stratégie RSI Mean Reversion
# À copier dans l'IDE QuantConnect

from AlgorithmImports import *

class RSIMeanReversion(QCAlgorithm):
    """
    Stratégie de retour à la moyenne basée sur le RSI.
    
    Achète lorsque RSI < 30 (survente)
    Vend lorsque RSI > 70 (surachat)
    """
    
    def Initialize(self):
        # Configuration de base
        self.SetStartDate(2020, 1, 1)
        self.SetEndDate(2023, 12, 31)
        self.SetCash(100000)
        
        # Ajouter SPY avec résolution journalière
        self.symbol = self.AddEquity("SPY", Resolution.Daily).Symbol
        
        # Créer l'indicateur RSI(14)
        # 14 est la période standard pour le RSI
        self.rsi = self.RSI(self.symbol, 14)
        
        # Warm-up de 14 jours pour initialiser le RSI
        self.SetWarmUp(14)
        
        self.Log("RSI Mean Reversion strategy initialized")
        self.Log(f"Backtest period: {self.StartDate} to {self.EndDate}")
    
    def OnData(self, data):
        # 1. Ignorer la période de warm-up
        if self.IsWarmingUp:
            return
        
        # 2. Vérifier que le RSI est prêt
        if not self.rsi.IsReady:
            return
        
        # 3. Obtenir la valeur actuelle du RSI
        rsi_value = self.rsi.Current.Value
        
        # 4. Obtenir le prix actuel
        price = data[self.symbol].Close
        
        # 5. Visualiser le RSI dans les graphiques
        self.Plot("RSI", "RSI(14)", rsi_value)
        self.Plot("RSI", "Overbought", 70)
        self.Plot("RSI", "Oversold", 30)
        
        # 6. Logique de trading
        
        # Signal d'ACHAT : RSI < 30 (survente) et pas déjà investi
        if rsi_value < 30 and not self.Portfolio.Invested:
            # Acheter 100% du capital disponible
            self.SetHoldings(self.symbol, 1.0)
            self.Log(f"BUY: RSI={rsi_value:.2f} | Price=${price:.2f}")
        
        # Signal de VENTE : RSI > 70 (surachat) et actuellement investi
        elif rsi_value > 70 and self.Portfolio.Invested:
            # Liquider toute la position
            self.Liquidate(self.symbol)
            self.Log(f"SELL: RSI={rsi_value:.2f} | Price=${price:.2f}")
    
    def OnEndOfAlgorithm(self):
        # Afficher les résultats finaux
        final_value = self.Portfolio.TotalPortfolioValue
        initial_value = self.StartingCash
        total_return = ((final_value - initial_value) / initial_value) * 100
        
        self.Log(f"")
        self.Log(f"="*50)
        self.Log(f"BACKTEST SUMMARY")
        self.Log(f"="*50)
        self.Log(f"Initial Capital: ${initial_value:,.2f}")
        self.Log(f"Final Portfolio Value: ${final_value:,.2f}")
        self.Log(f"Total Return: {total_return:.2f}%")
        self.Log(f"="*50)

### Explication détaillée du code

#### Dans Initialize()

1. **Dates et capital** : Backtest sur 4 ans (2020-2023) avec 100k$ de capital initial
2. **Security** : SPY (ETF S&P 500) avec résolution Daily
3. **Indicateur RSI(14)** : Période standard de 14 jours
4. **Warm-up** : 14 jours pour initialiser le RSI avant le début du backtest

#### Dans OnData()

1. **Vérifications préliminaires** :
   - Skip la période de warm-up (`IsWarmingUp`)
   - Vérifier que le RSI est prêt (`rsi.IsReady`)

2. **Récupération des données** :
   - Valeur du RSI : `rsi.Current.Value`
   - Prix de clôture : `data[symbol].Close`

3. **Visualisation** :
   - Tracer le RSI avec les lignes de référence 30/70

4. **Logique de trading** :
   - **Achat** : RSI < 30 ET pas investi → `SetHoldings(symbol, 1.0)`
   - **Vente** : RSI > 70 ET investi → `Liquidate(symbol)`

#### Dans OnEndOfAlgorithm()

- Calcul et affichage du rendement total
- Résumé des performances

### Lancer dans QuantConnect cloud

Pour tester cette stratégie :

1. **Connectez-vous** à [QuantConnect](https://www.quantconnect.com/)
2. **Créez un nouveau projet** : "New Project" → "Python Algorithm"
3. **Copiez le code** ci-dessus dans `main.py`
4. **Lancez le backtest** : Cliquez sur "Backtest"
5. **Analysez les résultats** :
   - Onglet "Overview" : Rendement total, Sharpe ratio, Drawdown
   - Onglet "Charts" : Graphique RSI, Equity curve
   - Onglet "Orders" : Tous les trades exécutés
   - Onglet "Logs" : Messages de self.Log()

### Analyser les résultats

Une fois le backtest terminé, observez :

| Métrique | Description | Valeur typique |
|----------|-------------|----------------|
| **Total Return** | Rendement total sur la période | Variable selon marché |
| **Sharpe Ratio** | Rendement ajusté au risque | > 1 = bon, > 2 = excellent |
| **Max Drawdown** | Plus grosse perte depuis un pic | < 20% = acceptable |
| **Win Rate** | % de trades gagnants | 50-60% typique pour mean reversion |
| **Number of Trades** | Nombre total de trades | Dépend de la volatilité |

**Questions à se poser** :
- La stratégie bat-elle le buy-and-hold de SPY ?
- Y a-t-il des périodes où elle performe mal ?
- Le Sharpe ratio justifie-t-il le risque ?
- Combien de trades sont exécutés par an ?

## 6. Méthodes QCAlgorithm Essentielles

### Méthodes de trading

Voici les méthodes les plus utilisées pour exécuter des ordres :

| Méthode | Signature | Usage | Exemple |
|---------|-----------|-------|--------|
| **SetHoldings** | `SetHoldings(symbol, percentage)` | Acheter/vendre pour atteindre X% du portfolio | `SetHoldings(spy, 0.5)` = 50% en SPY |
| **Liquidate** | `Liquidate(symbol)` | Vendre toute la position d'un actif | `Liquidate(spy)` |
| **MarketOrder** | `MarketOrder(symbol, quantity)` | Ordre au marché (exécution immédiate) | `MarketOrder(spy, 100)` = acheter 100 actions |
| **LimitOrder** | `LimitOrder(symbol, quantity, limit_price)` | Ordre limite (exécution si prix atteint) | `LimitOrder(spy, 100, 450)` |
| **StopMarketOrder** | `StopMarketOrder(symbol, quantity, stop_price)` | Stop loss (vendre si prix descend) | `StopMarketOrder(spy, -100, 440)` |

### Différences entre les types d'ordres

#### SetHoldings vs MarketOrder

```python
# SetHoldings - Spécifie un % du portfolio
self.SetHoldings(self.symbol, 0.8)  # Acheter pour avoir 80% du portfolio en SPY
# QuantConnect calcule automatiquement le nombre d'actions à acheter/vendre

# MarketOrder - Spécifie un nombre d'actions
self.MarketOrder(self.symbol, 100)  # Acheter exactement 100 actions
```

**Conseil** : Utilisez `SetHoldings` pour la simplicité, `MarketOrder` pour un contrôle précis.

#### MarketOrder vs LimitOrder

```python
# MarketOrder - Exécution immédiate au meilleur prix disponible
self.MarketOrder(self.symbol, 100)
# Avantage: Garantit l'exécution
# Inconvénient: Pas de contrôle sur le prix (slippage possible)

# LimitOrder - Exécution uniquement si prix <= limite (pour achat)
self.LimitOrder(self.symbol, 100, 450)
# Avantage: Contrôle du prix maximum
# Inconvénient: Peut ne jamais s'exécuter
```

### Exemples d'utilisation des méthodes

In [ ]:
# Exemples d'utilisation des méthodes de trading
# Code informatif pour QuantConnect IDE

from AlgorithmImports import *

class TradingMethodsExample(QCAlgorithm):
    
    def Initialize(self):
        self.SetStartDate(2020, 1, 1)
        self.SetCash(100000)
        self.symbol = self.AddEquity("SPY", Resolution.Daily).Symbol
    
    def OnData(self, data):
        price = data[self.symbol].Close
        
        # EXEMPLE 1: SetHoldings - Acheter pour 50% du portfolio
        if not self.Portfolio.Invested:
            self.SetHoldings(self.symbol, 0.5)
            self.Log(f"SetHoldings: Acheté pour 50% du portfolio")
        
        # EXEMPLE 2: MarketOrder - Acheter 100 actions au marché
        # self.MarketOrder(self.symbol, 100)
        # self.Log(f"MarketOrder: Acheté 100 actions à ~${price:.2f}")
        
        # EXEMPLE 3: LimitOrder - Acheter 100 actions si prix <= 450$
        # self.LimitOrder(self.symbol, 100, 450)
        # self.Log(f"LimitOrder: Ordre d'achat de 100 actions à 450$ max")
        
        # EXEMPLE 4: StopMarketOrder - Vendre si prix descend sous 440$
        # if self.Portfolio[self.symbol].Quantity > 0:
        #     quantity = -self.Portfolio[self.symbol].Quantity
        #     self.StopMarketOrder(self.symbol, quantity, 440)
        #     self.Log(f"StopMarketOrder: Stop loss à 440$")
        
        # EXEMPLE 5: Liquidate - Vendre toute la position
        # if self.Portfolio.Invested:
        #     self.Liquidate(self.symbol)
        #     self.Log(f"Liquidate: Position fermée à ${price:.2f}")

### Méthodes de logging et visualisation

| Méthode | Usage | Exemple |
|---------|-------|--------|
| **Log** | Message dans les résultats du backtest | `self.Log("Trade executed")` |
| **Debug** | Message dans la console de debug | `self.Debug(f"Price: {price}")` |
| **Plot** | Créer un graphique personnalisé | `self.Plot("Chart", "Series", value)` |
| **Error** | Logger une erreur | `self.Error("Something went wrong")` |

#### Exemple de visualisation avancée

```python
def OnData(self, data):
    price = data[self.symbol].Close
    
    # Graphique 1: Prix et moyennes mobiles
    self.Plot("Price Analysis", "SPY Price", price)
    if self.sma_20.IsReady:
        self.Plot("Price Analysis", "SMA(20)", self.sma_20.Current.Value)
    if self.sma_50.IsReady:
        self.Plot("Price Analysis", "SMA(50)", self.sma_50.Current.Value)
    
    # Graphique 2: RSI avec zones
    if self.rsi.IsReady:
        self.Plot("Momentum", "RSI", self.rsi.Current.Value)
        self.Plot("Momentum", "Overbought", 70)
        self.Plot("Momentum", "Oversold", 30)
    
    # Graphique 3: Position size
    holdings = self.Portfolio[self.symbol].Quantity
    self.Plot("Portfolio", "SPY Holdings", holdings)
```

### Méthodes d'accès au Portfolio

```python
# Vérifier si on a une position
is_invested = self.Portfolio.Invested
has_spy = self.Portfolio[self.symbol].Invested

# Quantité détenue
quantity = self.Portfolio[self.symbol].Quantity

# Valeur de la position
holding_value = self.Portfolio[self.symbol].HoldingsValue

# Prix moyen d'achat
avg_price = self.Portfolio[self.symbol].AveragePrice

# Profit/loss non réalisé
unrealized_pnl = self.Portfolio[self.symbol].UnrealizedProfit

# Valeur totale du portfolio
total_value = self.Portfolio.TotalPortfolioValue

# Cash disponible
cash = self.Portfolio.Cash
```

## 7. Conclusion

### Récapitulatif

Dans ce notebook, nous avons exploré les fondamentaux de la plateforme QuantConnect :

#### 1. QCAlgorithm Lifecycle

- **Initialize()** : Configuration initiale (1 fois)
- **OnData()** : Logique de trading (à chaque nouvelle donnée)
- **OnEndOfAlgorithm()** : Finalisation (1 fois)

#### 2. Gestion des Securities

- Ajouter des actifs avec `AddEquity()`, `AddCrypto()`, `AddForex()`
- Utiliser des objets `Symbol` (pas des strings)
- Configurer le warm-up avec `SetWarmUp()`

#### 3. Indicateurs Techniques

- Bibliothèque complète : SMA, EMA, RSI, MACD, BB, etc.
- Toujours vérifier `indicator.IsReady`
- Accéder aux valeurs avec `indicator.Current.Value`

#### 4. Stratégie RSI Mean Reversion

- Exemple complet d'une stratégie fonctionnelle
- Achat en zone de survente (RSI < 30)
- Vente en zone de surachat (RSI > 70)

#### 5. Méthodes Essentielles

- **Trading** : `SetHoldings()`, `Liquidate()`, `MarketOrder()`, `LimitOrder()`
- **Logging** : `Log()`, `Debug()`, `Plot()`
- **Portfolio** : `Portfolio.Invested`, `Portfolio[symbol].Quantity`

### Tableau récapitulatif des méthodes principales

| Catégorie | Méthode | Usage |
|-----------|---------|-------|
| **Configuration** | `SetStartDate()`, `SetEndDate()`, `SetCash()` | Définir période et capital |
| **Securities** | `AddEquity()`, `AddCrypto()`, `AddForex()` | Ajouter des actifs |
| **Indicateurs** | `SMA()`, `EMA()`, `RSI()`, `MACD()`, `BB()` | Créer des indicateurs |
| **Trading** | `SetHoldings()`, `Liquidate()`, `MarketOrder()` | Exécuter des ordres |
| **Logging** | `Log()`, `Debug()`, `Plot()` | Debugger et visualiser |
| **Portfolio** | `Portfolio.Invested`, `Portfolio[symbol].Quantity` | Consulter l'état |

### Prochaine étape : QC-Py-03-Data-Management

Dans le prochain notebook, nous approfondirons la gestion des données avec :

- **History API** : Récupérer des données historiques
- **Consolidators** : Agréger des données (ex: minute → hour)
- **Scheduled Events** : Exécuter du code à des heures précises
- **Universe Selection** : Trader plusieurs actifs dynamiquement

### Exercice suggéré

Modifiez la stratégie RSI Mean Reversion pour :

1. **Ajuster les seuils** : Testez RSI < 25 et RSI > 75
2. **Ajouter un stop loss** : Vendez si perte > 5%
3. **Tester un autre actif** : Remplacez SPY par QQQ (Nasdaq 100)
4. **Combiner avec une moyenne mobile** : N'achetez que si prix > SMA(50)
5. **Créer une stratégie MACD crossover** :
   - Achetez quand MACD croise au-dessus du signal
   - Vendez quand MACD croise en-dessous du signal

### Ressources supplémentaires

- [QCAlgorithm API Reference](https://www.quantconnect.com/docs/v2/writing-algorithms/api-reference)
- [Supported Indicators](https://www.quantconnect.com/docs/v2/writing-algorithms/indicators/supported-indicators/overview)
- [Strategy Library](https://www.quantconnect.com/tutorials/strategy-library) - Exemples de stratégies complètes
- [Documentation officielle](https://www.quantconnect.com/docs/v2)

---

**Félicitations !** Vous maîtrisez maintenant les fondamentaux de la plateforme QuantConnect. Vous êtes prêt à créer vos propres stratégies de trading algorithmique.